In [4]:
from convokit import Corpus, download
corpus = Corpus(filename=download("movie-corpus"))

In [5]:
# 使用len()函数来获取对话数和话语数
print("总话语数:", len(corpus.utterances))
print("总对话数（会话数）:", len(corpus.conversations))

# 访问并打印第一个对话的详细内容
for conversation_id in corpus.conversations:
    conversation = corpus.get_conversation(conversation_id)
    print(f"对话ID: {conversation_id}")
    for utterance in conversation.iter_utterances():
        print(f"{utterance.speaker.id}: {utterance.text}")
    break  # 只打印第一个对话，避免输出太多内容

总话语数: 304713
总对话数（会话数）: 83097
对话ID: L1044
u0: They do not!
u2: They do to!


In [24]:
conversations_texts = []

# 迭代前1000个对话
for i, conversation_id in enumerate(corpus.conversations):
    if i >= 10000:  # 只处理前1000条对话
        break
    conversation = corpus.get_conversation(conversation_id)
    # 收集每个对话中的话语文本
    conversation_text = ' '.join([utterance.text for utterance in conversation.iter_utterances()])
    conversations_texts.append(conversation_text)

In [14]:
import tiktoken
# 创建一个基本的单词标记器

encoding = tiktoken.get_encoding("cl100k_base")

In [25]:
tokenized_text = encoding.encode("".join(conversations_texts))

print(len(tokenized_text))

502498


In [ ]:
train_idex = int(len(tokenized_text) * 0.8)
train_data = tokenized_text[:train_idex]
valid_data = tokenized_text[train_idex:]